<a href="https://colab.research.google.com/github/AkshayDawkhar/IPL_predictionsDjango/blob/main/match_winner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder

In [22]:
data = pd.read_csv("/content/drive/MyDrive/ml/matches.csv")

Reading CSV from drive

In [23]:
data = data.iloc[:,:-1]
data.dropna(inplace=True)

 removing umpire 3
Having null value

In [24]:
data.replace(['Deccan Chargers','Delhi Daredevils','Rising Pune Supergiant','Rising Pune Supergiants'],['Delhi Capitals','Sunrisers Hyderabad','Pune Warriors','Pune Warriors'],inplace=True)

Replacing the team names

In [25]:
venuelab=LabelEncoder()
venue=data['venue']
venue=venuelab.fit_transform(venue)
le_name_mapping = dict(zip(venuelab.classes_, venuelab.transform(venuelab.classes_)))
# print(le_name_mapping)

Labeling vanue and saving the dictionary

In [26]:
data.drop(['win_by_runs','win_by_wickets','id','season','city','venue','date','player_of_match','umpire1','umpire2','dl_applied','result'], axis=1,inplace= True)

Droping unnecessary columes

In [27]:
data['toss_decision'] = data['toss_decision'].replace(['field','bat'],[1,0])
x = data.drop(["winner"], axis=1)
y = data["winner"]

for i in range(len(y)):
  #  print(x.iloc[i,0]==y.iloc[i])
   if x.iloc[i,0]== y.iloc[i]:
      y.iloc[i] = 0
  #  elif x.iloc[i,1]== y.iloc[i]:
   else:
      y.iloc[i] = 1
   if x.iloc[i,0]==  x.iloc[i,2]:
      x.iloc[i,2] = 0
  #  elif x.iloc[i,1]== y.iloc[i]:
   else:
      x.iloc[i,2] = 1
  #  if  x.iloc[i,3] == 'field':
# x.iloc[1,1] == 'as'
# x.iloc[1,1]
toss_winn=x.iloc[:,2]
toss_deci=x.iloc[:,3]
x.drop(['toss_winner','toss_decision'],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [28]:
X = pd.get_dummies(x, ["team1","team2"])
X['venue']=venue
X['toss_winner']=toss_winn
X['toss_decision']=toss_deci
le = LabelEncoder()
y = le.fit_transform(y)

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

In [30]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=400,min_samples_split=3,
                               max_features = "auto")
model.fit(X, y)

RandomForestClassifier(min_samples_split=3, n_estimators=400)

In [31]:
def pr_win(team1,team2,tosw,tosd,stem):
  arr=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  teamcode={'CSK':0, 'DD':1, 'GL':2, 'KBKP':3, 'KTK':4, 'KKR':5, 'MI':6, 'PW':7, 'RR':8, 'RCB':9,
       'SRH':10,'field':1, 'bat':0}
  arr[23]=tosw
  arr[24]=tosd
  arr[22]=le_name_mapping[stem]
  arr[(11+teamcode[team2])]=1
  arr[(teamcode[team1])]=1
  pred=np.array(arr)
  ass=model.predict(pred.reshape(1, -1))
  if ass == 0:
    print(team1)
  else:
    print(team2)

pr_win('RCB','MI',0,1,'Wankhede Stadium')

MI


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [42]:
le_name_mapping

{'ACA-VDCA Stadium': 0,
 'Barabati Stadium': 1,
 'Brabourne Stadium': 2,
 'Buffalo Park': 3,
 'De Beers Diamond Oval': 4,
 'Dr DY Patil Sports Academy': 5,
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium': 6,
 'Eden Gardens': 7,
 'Feroz Shah Kotla': 8,
 'Feroz Shah Kotla Ground': 9,
 'Green Park': 10,
 'Himachal Pradesh Cricket Association Stadium': 11,
 'Holkar Cricket Stadium': 12,
 'IS Bindra Stadium': 13,
 'JSCA International Stadium Complex': 14,
 'Kingsmead': 15,
 'M Chinnaswamy Stadium': 16,
 'M. A. Chidambaram Stadium': 17,
 'M. Chinnaswamy Stadium': 18,
 'MA Chidambaram Stadium, Chepauk': 19,
 'Maharashtra Cricket Association Stadium': 20,
 'Nehru Stadium': 21,
 'New Wanderers Stadium': 22,
 'Newlands': 23,
 'OUTsurance Oval': 24,
 'Punjab Cricket Association IS Bindra Stadium, Mohali': 25,
 'Punjab Cricket Association Stadium, Mohali': 26,
 'Rajiv Gandhi International Stadium, Uppal': 27,
 'Rajiv Gandhi Intl. Cricket Stadium': 28,
 'Sardar Patel Stadium, Motera': 29,
 '

In [33]:
toss_x=np.array(X.drop(['toss_winner','toss_decision'],axis=1))
toss_y=X['toss_winner'].astype('int')

from sklearn.model_selection import train_test_split
tossx_train, tossx_test, tossy_train, tossy_test = train_test_split(toss_x, toss_y, train_size = 0.8)

In [34]:
modeltoss_win=RandomForestClassifier(n_estimators=400,min_samples_split=3,
                               max_features = "auto")
modeltoss_win.fit(toss_x,toss_y)
# toss_pred=modeltoss_win.predict(tossx_test)

RandomForestClassifier(min_samples_split=3, n_estimators=400)

In [35]:
from sklearn.neural_network import MLPClassifier
#Initializing the MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=400,activation = 'relu',solver='adam',random_state=1)
classifier.fit(tossx_train, tossy_train)
Rtoss = classifier.predict(tossx_test)

In [36]:
def toss_win(team1,team2,stem):
  arr=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  teamcode={'CSK':0, 'DD':1, 'GL':2, 'KBKP':3, 'KTK':4, 'KKR':5, 'MI':6, 'PW':7, 'RR':8, 'RCB':9,
       'SRH':10,'field':1, 'bat':0}
  arr[22]=le_name_mapping[stem]
  arr[(11+teamcode[team2])]=1
  arr[(teamcode[team1])]=1
  pred=np.array(arr)
  ass=classifier.predict(pred.reshape(1, -1))
  if ass == 0:
    print(team1)
  else:
    print(team2)
toss_win('MI','CSK','Holkar Cricket Stadium')

CSK


In [50]:
from joblib import dump
dump(classifier,'modeltoswin.joblib')

['modeltoswin.joblib']

In [45]:
toss_dicy=X.iloc[:,-1]
toss_dicX=X.drop(['toss_decision'],axis=1)
toss_dicy

0      1
1      1
2      1
3      1
5      1
      ..
750    1
751    1
752    0
754    1
755    0
Name: toss_decision, Length: 743, dtype: int64

In [46]:
model_tossdic=RandomForestClassifier(n_estimators=400,min_samples_split=3,max_features = "auto")

In [47]:
model_tossdic.fit(toss_dicX,toss_dicy)
toss_dicX.to_csv('a.csv')
m=model_tossdic.predict([[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,30,1]])
m

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


array([1])

In [48]:
toss_dicX

,team1_Chennai Super Kings,team1_Delhi Capitals,team1_Gujarat Lions,team1_Kings XI Punjab,team1_Kochi Tuskers Kerala,team1_Kolkata Knight Riders,team1_Mumbai Indians,team1_Pune Warriors,team1_Rajasthan Royals,team1_Royal Challengers Bangalore,...,team2_Kings XI Punjab,team2_Kochi Tuskers Kerala,team2_Kolkata Knight Riders,team2_Mumbai Indians,team2_Pune Warriors,team2_Rajasthan Royals,team2_Royal Challengers Bangalore,team2_Sunrisers Hyderabad,venue,toss_winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,27,1
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,20,1
2,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,30,1
3,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,12,1
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,27,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,13,1
751,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,39,1
752,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,17,0
754,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [53]:
def toss_dic(team1,team2,toss_win,stem):
  arr=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  teamcode={'CSK':0, 'DD':1, 'GL':2, 'KBKP':3, 'KTK':4, 'KKR':5, 'MI':6, 'PW':7, 'RR':8, 'RCB':9,
       'SRH':10,'field':1, 'bat':0}
  arr[23]=le_name_mapping[stem]    
  # arr[22]=le_name_mapping[stem]
  arr[(11+teamcode[team2])]=1
  arr[(teamcode[team1])]=1
  pred=np.array(arr)
  ass=model_tossdic.predict(pred.reshape(1, -1))
  print(arr)
  if ass == 0:
    print(team1)
  else:
    print(team2)
toss_dic('MI','CSK',1,'Holkar Cricket Stadium')

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12]
CSK


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [51]:
from joblib import dump
dump(model_tossdic,'modeltosdic.joblib')

['modeltosdic.joblib']